In [1]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
data = pd.read_csv('file/bquxjob_157d5c5d_1941367a86e.csv')

# Display the first few rows of the DataFrame to understand its structure
data.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Campania,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2024-12-14,BOOMIT_PEIG_EC_GADS_ADQUISI_ANDROID_(PRIMTARJ)...,Google Ads,424.01,120,431,0.278422
1,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,31.16,9,27,0.333333
2,2024-12-14,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,58.88,19,45,0.422222
3,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(BIG-PROMO)_B...,Meta,10.31,0,0,0.000000
4,2024-12-14,BOOMIT_PEIG_EC_META_TRAFICO_GRAL_(PROMO-PEIGO-...,Meta,19.21,0,0,0.000000


In [2]:
# Convert 'Fecha' to datetime format for easier manipulation
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Find the most recent date in the dataset
most_recent_date = data['Fecha'].max()

# Determine the two periods for comparison
period_end = most_recent_date
period_start = period_end - pd.Timedelta(days=6)
previous_period_end = period_start - pd.Timedelta(days=1)
previous_period_start = previous_period_end - pd.Timedelta(days=6)

# Filter the data for the two periods
current_period_data = data[(data['Fecha'] >= period_start) & (data['Fecha'] <= period_end)]
previous_period_data = data[(data['Fecha'] >= previous_period_start) & (data['Fecha'] <= previous_period_end)]

# Remove platforms with zero or near-zero investment
current_period_data = current_period_data[current_period_data['inversion'] > 0.1]
previous_period_data = previous_period_data[previous_period_data['inversion'] > 0.1]

# Display the date ranges for the periods
period_start, period_end, previous_period_start, previous_period_end

(Timestamp('2024-12-22 00:00:00'),
 Timestamp('2024-12-28 00:00:00'),
 Timestamp('2024-12-15 00:00:00'),
 Timestamp('2024-12-21 00:00:00'))

In [4]:
# Group by platform and calculate the required metrics for each period
def calculate_metrics(data):
    metrics = data.groupby('Plataforma').agg({
        'evento_objetivo': 'sum',
        'inversion': 'sum',
        'Instalaciones': 'sum'
    }).reset_index()
    metrics['costo_medio_evento'] = metrics['inversion'] / metrics['evento_objetivo']
    metrics['costo_medio_instalacion'] = metrics['inversion'] / metrics['Instalaciones']
    metrics['cvr_medio'] = metrics['evento_objetivo'] / metrics['Instalaciones']
    return metrics

current_metrics = calculate_metrics(current_period_data)
previous_metrics = calculate_metrics(previous_period_data)

# Calculate percentage changes between the two periods
comparison = current_metrics.set_index('Plataforma').join(previous_metrics.set_index('Plataforma'), lsuffix='_current', rsuffix='_previous')
comparison['var_costo_medio_evento'] = (comparison['costo_medio_evento_current'] - comparison['costo_medio_evento_previous']) / comparison['costo_medio_evento_previous'] * 100
comparison['var_eventos'] = (comparison['evento_objetivo_current'] - comparison['evento_objetivo_previous']) / comparison['evento_objetivo_previous'] * 100
comparison['var_costo_medio_instalacion'] = (comparison['costo_medio_instalacion_current'] - comparison['costo_medio_instalacion_previous']) / comparison['costo_medio_instalacion_previous'] * 100
comparison['var_instalaciones'] = (comparison['Instalaciones_current'] - comparison['Instalaciones_previous']) / comparison['Instalaciones_previous'] * 100
comparison['var_inversion'] = (comparison['inversion_current'] - comparison['inversion_previous']) / comparison['inversion_previous'] * 100

# Display the comparison results
comparison[['var_costo_medio_evento', 'var_eventos', 'var_costo_medio_instalacion', 'var_instalaciones', 'var_inversion']]

,var_costo_medio_evento,var_eventos,var_costo_medio_instalacion,var_instalaciones,var_inversion
Plataforma,,,,,
Google Ads,-43.766718,68.767123,-29.990938,35.558583,-5.096707
Meta,-17.024172,19.047619,-7.041695,6.263499,-1.219252
TikTok Ads,-39.401496,65.020576,-26.608370,36.255319,0.000000
